In [1]:
from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import folium
import json
import os
from branca.colormap import linear
import branca.colormap

## Data

We'll use the same "us-states" data from last time. For the choropleth, we'll use the "US_Unemployment_Oct2012.csv" dataset that contains unemployment rates by State for October 2012.

In [2]:
# US States GeoJSON file
us_states = os.path.join('data', 'us-states.json')
geo_json_data = json.load(open(us_states))

# Unemployment Data
US_Unemployment_Oct2012 = os.path.join('data', 'US_Unemployment_Oct2012.csv')
unemployment = pd.read_csv(US_Unemployment_Oct2012)

## Base Map Review

Plot a basemap of the United States, overlay it with the "us_states" GeoJSON file, and customize the color scheme.

In [3]:
m = folium.Map([43, -100], zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': 'yellow',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

m

## Choropleth

Choropleths are a mapping technique that are popular among data scientists because they use color to display statistical information. Generally, choropleths shade geographic regions by intensity of some measure (in our case, unemployment rates).

**Step 1**:

Before we start shading states by their unemployment rates, we need to decide on a color scale! 

Consult the branca documentation, and scroll down to "_schemes": https://pypkg.com/pypi/branca/f/branca/colormap.py

Then, create an object called colormap that uses "linear. .scale()" to create a sliding color scale that ranges from the minimum unemployment to the maximum.

In [4]:
colormap = linear.BuGn.scale(
    unemployment.Unemployment.min(),
    unemployment.Unemployment.max())

colormap

**Step 2**:

Next, we need to convert our unemployment into a dictionary, which will be passed into the folium.GeoJson argument. Take a peak at the dataframe (note that we loaded it as a pandas dataframe instead of a datascience Table) by using the "head" method. Then, create an object called "unemployment_dict" by using the "set_index" method.

In [5]:
unemployment.head(10)

,State,Unemployment
0,AL,7.1
1,AK,6.8
2,AZ,8.1
3,AR,7.2
4,CA,10.1
5,CO,7.7
6,CT,8.4
7,DE,7.1
8,FL,8.2
9,GA,8.8


In [6]:
unemployment_dict = unemployment.set_index('State')['Unemployment']

**Step 3**:

We're ready to plot! Use folium.Map to create a base map of the U.S., then folium.GeoJson to overlay the geo_json_data and add color. Hint: For the 'fillColor' argument, try something like this: 

'fillColor': colormap(unemployment_dict[feature['id']]

In [7]:
m = folium.Map([43, -100], zoom_start=4)

folium.GeoJson(
    geo_json_data,
    name='unemployment',
    style_function=lambda feature: {
        'fillColor': colormap(unemployment_dict[feature['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.LayerControl().add_to(m)

m

To wrap up, let's add a color legend. Use "colormap.caption" and "colormap.add_to()" to add the legend to the map, and call it again.

In [8]:
colormap.caption = 'Unemployment color scale'
colormap.add_to(m)

m

#### Question 1

What patterns do you notice in the variation in state unemployment rates in 2012? What do you think might help explain some of these differences?

## Fun With Colors

Let's experiment with a few different options in the "colormap" module. First, try filling in the code below to customize a color pallette. Fill in three colors, and indices for the color breaks:

In [9]:
step = branca.colormap.StepColormap(
    ['green', 'yellow', 'red'],
    vmin=3, vmax=10,
    index=[3, 4, 8, 10],
    caption='step'
)

step

Now replot the map of U.S. unemployment with this new color scheme!

In [10]:
m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': step(unemployment_dict[feature['id']]),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

m

Use "branca.colormap.linear" to explore different color scale gradients. Experiment with choropleths that use different color schemes!

In [11]:
branca.colormap.linear

BuGn
BuPu
GnBu
OrRd
PuBu
PuBuGn


In [12]:
colormap = linear.Dark2.scale(
    unemployment.Unemployment.min(),
    unemployment.Unemployment.max())

m = folium.Map([43, -100], zoom_start=4)

folium.GeoJson(
    geo_json_data,
    name='unemployment',
    style_function=lambda feature: {
        'fillColor': colormap(unemployment_dict[feature['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

folium.LayerControl().add_to(m)

colormap.caption = 'Unemployment color scale'
colormap.add_to(m)

m

Finally, let's create a map that allows you to highlight states and zoom in on them. Use the "highlight" argument to add this feature.

In [13]:
state_data = pd.read_csv(US_Unemployment_Oct2012)

m = folium.Map(location=[48, -102], zoom_start=3)
m.choropleth(
    geo_data=us_states,
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)',
    highlight=True
)

m